# Taller: Simulación en Python


##A. Considerando el sistema eléctrico de la forma:

$f(t): L \ddot{I}(t) + R\dot{I}(t) + \frac{1}{C} I(t) = E(t)$

con las constantes:

$L = 0,001H$

$R = 0,02 \Omega$

$C = 2F$

$E = \sin(t)$

Suponiendo condiciones iniciales iguales a cero y un tiempo de simulación de 15s.

1. La función de transferencia ($H(s) = \frac{\text{Salida}}{\text{Entrada}}$) se obtiene al despejar la ecuación en el dominio de $\mathscr{L}$aplace:


$\mathscr{L}\{{f(t)}\} : Ls^2I(s) + RsI(s) + \frac{1}{C}I(s) = E(s)$

$I(s)[Ls^2 + Rs + \frac{1}{C}] = E(s)$

$\therefore H(s) = \frac{I(s)}{E(s)} = \frac{1}{Ls^2 + Rs + \frac{1}{C}}$

2. A partir de aquí, se puede resolver el sistema usando este $H(s)$



In [ ]:
from scipy.integrate import solve_ivp
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt
import scipy.signal as signal
import numpy as np

In [ ]:
# Constantes de la ecuación.
L = 0.001 #H
R = 0.02 #Ohm
C = 2 #F

# Entrada (Fuente de voltaje E).
start = 0
stop = 15
step = 1e-3
time = np.arange(start, stop + step, step)
E = np.sin(time)

In [ ]:
# Coeficientes del numerador y denominador de la función de transferencia.
num = [1]
den = [L, R, 1/C]

# Creación del sistema.
sys_1 = signal.TransferFunction(num, den)
t_out1, i_out1, _ = signal.lsim(sys_1, E, T = time)

# Graficar la solución.
plt.figure(figsize=(10, 5))
plt.plot(t_out1, i_out1, color = 'black')
plt.xlim(0, 15)
plt.xlabel('Tiempo [s]')
plt.ylabel('Corriente [A]')
plt.title('Respuesta a una fuente de voltaje sinusoidal')
plt.grid(True)
plt.show()


También se puede resolver mediante ecuaciones diferenciales. Para ello, se rescribe la ecuación como si fuera de primer orden:

$I_1(t) = I(t)$

$I_2(t) = \dot{I} (t) \therefore I_2 (t) = \dot{I_1} (t)$

$\dot{I_2} (t) = \ddot{I} (t)$

$\therefore L \ddot{I}(t) + R\dot{I}(t) + \frac{1}{C} I(t) = E(t) ⇒ L \dot{I_2} (t) + R I_2 (t) + \frac{1}{C} I_1 (t) = E (t)$

$\therefore \dot{I_2} (t) = \frac{E(t) - R I_2 (t) - \frac{1}{C} I_1 (t)}{L}$



In [ ]:
# Función que define el sistema de ecuaciones diferenciale a resolver.
def sys_2(t, y, R, L, C):
  '''
    t: Variable independiente de la función.
    y: Variable dependiente de la función.
    R, L, C: Constantes del sistema.
  '''

  # Se define la entrada para cada valor de t
  E_t = np.sin(t)

  # Planteando el sistema de ecuaciones
  i1, i2 = y # y[0]: Corriente, y[1]: Primera direvada de la corriente
  d_i1 = i2
  d_i2 = (E_t - R*i2 - (1/C)*i1)  / L

  return [d_i1, d_i2]

# Condiciones iniciales
y0 = [0, 0]
t_span = (start, stop)

In [ ]:
sol = solve_ivp(sys_2, t_span, y0, t_eval = time, args = (R, L, C))
t_out2 = sol.t
i_out2 = sol.y[0]
di_out2 = sol.y[1]
# Graficar
plt.figure(figsize=(10, 5))
plt.plot(t_out2, i_out2, color = 'black')
plt.xlim(0, 15)
plt.xlabel('Tiempo [s]')
plt.ylabel('Corriente [A]')
plt.title('Respuesta a una fuente de voltaje sinusoidal')
plt.grid(True)
plt.show()

Comparación entre los dos métodos

In [ ]:
plt.figure(figsize=(10, 6))
plt.suptitle('Respuesta de la corriente ante una entrada sinusoidal')

plt.subplot(2, 2, 1)
plt.plot(t_out1, i_out1, color = 'blue')
plt.xlim(0, 15)
plt.xlabel('Tiempo [s]')
plt.ylabel('Corriente [A]')
plt.title('Método de la función de transferencia')
plt.grid(True)

plt.subplot(2, 2, 2)
plt.plot(t_out2, i_out2, color = 'red', alpha=0.5)
plt.xlim(0, 15)
plt.xlabel('Tiempo [s]')
plt.ylabel('Corriente [A]')
plt.title('Método de las ecuaciones diferenciales')
plt.grid(True)

plt.subplot(2, 1, 2)
plt.plot(t_out1, i_out1, color = 'blue', label='Método de la función de transferencia')
plt.plot(t_out2, i_out2, color = 'red', alpha = 0.5, label='Método de las ecuaciones diferenciales')
plt.xlim(0, 15)
plt.xlabel('Tiempo [s]')
plt.ylabel('Corriente [A]')
plt.title('Comparación entre los dos métodos')
plt.grid(True)
plt.legend()

plt.tight_layout()
plt.show()

Se evidencia que, en este caso donde las condiciones iniciales son cero, ambos métodos arrojan resultados análogos para todo los valores de tiempo

Ahora se tomaran los datos de las constantes desde `parametros.txt`

In [ ]:
file_name = 'parametros.txt'
with open(file_name, 'r') as file:
  lines = file.readlines()                              # ['0.001\n', '0.02\n', '2']
  constants  = [float(line.strip()) for line in lines]  # [0.001, 0.02, 2.0]

L, R, C = constants

#################### MÉTODO DE LA FUNCIÓN DE TRANSFERENCIA #####################
# Entrada (Fuente de voltaje E).
start = 0
stop = 15
step = 1e-3
time = np.arange(start, stop + step, step)
E = np.sin(time)

# Coeficientes del numerador y denominador de la función de transferencia.
num = [1]
den = [L, R, 1/C]

# Creación del sistema.
sys_1 = signal.TransferFunction(num, den)
t_out1, i_out1, _ = signal.lsim(sys_1, E, T = time)

############# MÉTODO DE LA FUNCIÓN DE LAS ECUACIONES DIFERENCIALES #############
# Función que define el sistema de ecuaciones diferenciale a resolver.
def sys_2(t, y, R, L, C):
  '''
    t: Variable independiente de la función.
    y: Variable dependiente de la función.
    R, L, C: Constantes del sistema.
  '''

  # Se define la entrada para cada valor de t
  E_t = np.sin(t)

  # Planteando el sistema de ecuaciones
  i1, i2 = y # y[0]: Corriente, y[1]: Primera direvada de la corriente
  d_i1 = i2
  d_i2 = (E_t - R*i2 - (1/C)*i1)  / L

  return [d_i1, d_i2]

# Condiciones iniciales
y0 = [0, 0]
t_span = (start, stop)

sol = solve_ivp(sys_2, t_span, y0, t_eval = time, args = (R, L, C))
t_out2 = sol.t
i_out2 = sol.y[0]
di_out2 = sol.y[1]

################################### GRÁFICOS ###################################
plt.figure(figsize=(12, 4))
plt.suptitle('Respuesta de la corriente ante una entrada sinusoidal')

plt.subplot(1, 2, 1)
plt.plot(t_out1, i_out1, color = 'blue')
plt.xlim(0, 15)
plt.xlabel('Tiempo [s]')
plt.ylabel('Corriente [A]')
plt.title('Método de la función de transferencia')
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(t_out2, i_out2, color = 'red', alpha=0.5)
plt.xlim(0, 15)
plt.xlabel('Tiempo [s]')
plt.ylabel('Corriente [A]')
plt.title('Método de las ecuaciones diferenciales')
plt.grid(True)

plt.tight_layout()
plt.savefig('A-3.png')
plt.show()

Modificando el módelo para que la entrada E sea creada en un `.txt` y utilizada en la función de la simulación.

In [ ]:
# Crear la variable independiente
start = 0
stop = 15
step = 1e-3
time = np.arange(start, stop + step, step)

# Crear la función de entrada
E = np.sin(time)

# Crear una matriz con ambos valores
data = np.column_stack((time, E))

# Guardar la matriz como un archivo .txt
np.savetxt('Entrada.txt', data)

In [ ]:
# Lectura del archivo de texto
data = np.loadtxt("Entrada.txt")
t_txt = data[:, 0]
E_txt = data[:, 1]

# Creación de la función a partir de los datos
E_func = interp1d(t_txt, E_txt, fill_value = 'extrapolate')

In [ ]:
file_name = 'parametros.txt'
with open(file_name, 'r') as file:
  lines = file.readlines()                              # ['0.001\n', '0.02\n', '2']
  constants  = [float(line.strip()) for line in lines]  # [0.001, 0.02, 2.0]

L, R, C = constants

############# MÉTODO DE LA FUNCIÓN DE LAS ECUACIONES DIFERENCIALES #############
# Función que define el sistema de ecuaciones diferenciale a resolver.
def sys_2(t, y, R, L, C):
  '''
    t: Variable independiente de la función.
    y: Variable dependiente de la función.
    R, L, C: Constantes del sistema.
  '''

  # Se define la entrada para cada valor de t
  E_value = E_func(t)

  # Planteando el sistema de ecuaciones
  i1, i2 = y # y[0]: Corriente, y[1]: Primera direvada de la corriente
  d_i1 = i2
  d_i2 = (E_value - R*i2 - (1/C)*i1)  / L

  return [d_i1, d_i2]

# Condiciones iniciales
y0 = [0, 0]
t_span = (start, stop)

sol = solve_ivp(sys_2, t_span, y0, t_eval = time, args = (R, L, C))
t_out2 = sol.t
i_out2 = sol.y[0]
di_out2 = sol.y[1]

################################### GRÁFICOS ###################################
plt.figure(figsize=(12, 4))
plt.title('Respuesta de la corriente ante una entrada sinusoidal')
plt.plot(t_out2, i_out2, color = 'black')
plt.xlim(0, 15)
plt.xlabel('Tiempo [s]')
plt.ylabel('Corriente [A]')
plt.grid(True)

plt.tight_layout()
plt.show()

## Método de ecuaciones

In [ ]:
# CLASE
from scipy.integrate import solve_ivp
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt
import scipy.signal as signal
import numpy as np

# 1. Reescribir la ecuación como sistema de primer orden
# Se hace igualando X y derivada de X a variables lineales y creando relaciones entre estas.

# 2. Crear la función que define las ecuaciones a resolver
# a. Variable independiente
# b. Vector con la misma cantidad de elementos como ecuaciones equivalentes
# c. Constantes que de el problema (opcional)

def sys (t, y, m, c):
 # Creación de la fuerza de entrada
 F = np.where(t >= 5, 0.1, 0) # Consultar documentación
 # Descripción de las ecuaciones optenidas en el numeral 1
 x1, x2 = y # y[0] = x (posición), y[1] = dx/dt (velocidad)
 d_x1 = x2
 d_x2 = (1/m) * F - (c/m) * x2
 return [d_x1, d_x2]

In [ ]:
# 3. Realizar la simulación
# solve_ivp(fun, t_span, y0, method='RK45', t_eval=None, args=None)
# Función, tiempo de simulación, condiciones iniciales, método de integración, tiempo en que se almacena la solución, argumentos adicionales.

# Definición de constantes

m = 1 # masa
c = 0.8 # coeficiente de fricción
y0 = [0 , -1] # Condiciones iniciales, posición y velocidad (o sea x1 y x2)
Tm = 0.1 # Asumido para este problema, es el paso
total_time = 15
t_span = (0,total_time)
time = np.arange(0, total_time + Tm, Tm)

sol = solve_ivp(sys, t_span, y0, t_eval=time, args=(m,c))
# Entrega sol.t, que es el tiempo, sol.y[0], posición, y sol.y[1], velocidad.

In [ ]:
# Gráfica de la solución
plt.figure(figsize=(10, 3))
plt.subplot(1, 2, 1)
plt.plot(sol.t, sol.y[0])
plt.xlabel('Tiempo')
plt.ylabel('Posición')
plt.grid()
plt.subplot(1, 2, 2)
plt.plot(sol.t, sol.y[1])
plt.xlabel('Tiempo')
plt.ylabel('Velocidad')
plt.grid()

## Método por función de transferencia

In [ ]:
# 1. Obtener la función de transferencia
# Se pasan las ecuaciones a Laplace, por lo que no hay necesidad de reasignar variables

import scipy.signal as signal
# Coeficientes del numerador y denominador de la fn de transferencia
num = [1]
den = [m, c, 0]
# Entrada
F = np.where(time >= 5, 0.1, 0)
# Creación del sistema
sys2 = signal.TransferFunction(num, den)
t_out, y_out, _ = signal.lsim(sys2, F, T=time)
# Se le entrega el sistema, la entrada y el tiempo.

In [ ]:
# Gráfica de la solución
plt.figure(figsize=(10, 3))
plt.plot(t_out, y_out)
plt.xlabel('Tiempo')
plt.ylabel('Posición')
plt.grid()
plt.title("Método por función de transferencia")

### Comparación del primer método con condiciones iniciales en cero

In [ ]:
# m = 1
# c = 0.8
y0 = [0 , 0] # Condiciones iniciales, posición y velocidad (o sea x1 y x2)
# Tm = 0.1 # Asumido para este problema, es el paso
# total_time = 15
# t_span = (0,total_time)
# time = np.arange(0, total_time + Tm, Tm)

sol2 = solve_ivp(sys, t_span, y0, t_eval=time, args=(m,c))

In [ ]:
plt.figure(figsize=(10, 3))
plt.plot(sol.t, sol.y[0], label='y = [0, -1]')
plt.plot(sol2.t, sol2.y[0], label='y = [0, 0]')
plt.plot(t_out, y_out, "r--", label='Trans. func.')
plt.legend()
plt.xlabel('Tiempo')
plt.ylabel('Posición')
plt.grid()
plt.title("Comparación entre condiciones iniciales")